In [2]:
class Chromosome:
    '''
    Predstavlja jedan hromozom, odnosno jednu nisku.
    Niska je data genetskim kodom i ocenom prilagodjenosti 
    (u ovom primeru, broj karaktera date niske koji se poklapaju sa ciljnom niskom)
    '''
    def __init__(self, genetic_code, fitness):
        self.genetic_code = genetic_code
        self.fitness = fitness
    
    def __str__(self):
        return "{} = {}".format(''.join(self.genetic_code), self.fitness)

In [4]:
import random
import string

class GeneticAlgorithm:
    '''
    Klasa predstavlja implementaciju genetskog algoritma za resavanje problema pogadjanja niske.
    Koristi se:
    - Jednopoziciono ukrstanje sa nasumicnom tackom ukrstanja
    - Mutacija sa verovatnocom '_mutation_rate'
    - Ruletska selekcija ili turnirska selekcija u zavisnosti od parametra self.selection_type
    - Zamena generacije se vrsi tako sto se od jedinki izabranih pri selekciji ukrstanjem
        pravi celokupna nova generacija (pogledati u knjizi alternativne zamene generacija)
    '''
    def __init__(self, possible_gene_values, target):
        self.target = target                                     # Niska koja se pogadja
        self.gene_length = len(target)                           # Duzina niske koja se pogadja
        self.possible_gene_values = possible_gene_values         # Dozvoljene vrednosti koje mogu biti u genu
        self.num_gene_values = len(possible_gene_values)         # Duzina dozvoljenih vrednosti
        
        '''Parametri genetskog algoritma, eksperimentalno izabrani.'''
        self.generation_size = 5000            # Broj jedinki u jednoj generaciji
        self.chromosome_size = len(target)     # Duzina hromozoma
        self.reproduction_size = 1000          # Broj jedinki koji ucestvuje u reprodukciji    
        self.max_iterations = 1000             # Maksimalni dozvoljeni broj iteracija
        self.mutation_rate = 0.1               # Verovatnoca da se desi mutacija
        self.tournament_size = 10              # Velicina turnira
        
        # Vrsta selekcije, moze biti turnirska (tournament) ili ruletska (roulette)
        self.selection_type = 'tournament'      

    '''Vraca broj karaktera koji se u genu poklapaju sa pravom vrednoscu.'''
    def calculate_fitness(self, genetic_code):
        fitness_value = 0
        
        for i in range(self.gene_length):
            if genetic_code[i] == self.target[i]:
                fitness_value += 1
        
        return fitness_value
        
    '''Generise se _generation_size nasumicnih jedinki.'''
    def initial_population(self):
        init_population = []
        
        for i in range(self.generation_size):
            # generise se i-ti nasumicni genetski kod
            genetic_code = []
            
            for j in range(self.chromosome_size):
                # za svaki karakter u niski bira se neki nasumicni karakter iz liste dozvoljenih karaktera
                selected_value = random.choice(self.possible_gene_values)
                genetic_code.append(selected_value)
            
            # izracunava se prilagodjenost generisane niske
            fitness = self.calculate_fitness(genetic_code)
            new_chromosome = Chromosome(genetic_code, fitness)
            
            # dodaje se u populaciju
            init_population.append(new_chromosome)
            
        return init_population
    
    '''Funkcija za izbor hromozoma za reporodukciju'''
    def selection(self, chromosomes):
        selected = []
        
        # Bira se self.reproduction_size hromozoma za reprodukciju
        # Selekcija moze biti ruletska ili turnirska
        for i in range(self.reproduction_size):
            if self.selection_type == 'roulette':
                selected.append(self.roulette_selection(chromosomes))
            elif self.selection_type == 'tournament':
                selected.append(self.tournament_selection(chromosomes))
          
        # Vracaju se izabrani hromozomi za repodukciju
        return selected
    
    '''Bira jednu jedinku koristeci ruletsku selekciju. Ne vrsi normalizaciju
       i sortiranje po funkciji prilagodjenosti usled performansi.
    '''
    def roulette_selection(self, chromosomes):
        # suma svih prilagodjenosti
        total_fitness = sum([chromosome.fitness for chromosome in chromosomes])
        
        # bira se neka random vrednost, sluzi za 'imitiranje' slucajnosti 
        selected_value = random.randrange(0, total_fitness)
        
        current_sum = 0
        for i in range(self.generation_size):
            current_sum += chromosomes[i].fitness

            # vraca se prva jedinka koja ispuni uslov
            if current_sum > selected_value:
                return chromosomes[i]
     
    '''Bira jednu jedinku koristeci turnirsku selekciju. '''
    def tournament_selection(self, chromosomes):
        
        # bira se  self.tournament_size jediniki za turnir
        selected = random.sample(chromosomes, self.tournament_size)
        
        # pobednik je onaj sa najboljom prilagodjenosti
        winner = max(selected, key = lambda x: x.fitness)
        
        return winner
        
    '''Vrsi mutaciju nad hromozomom sa verovatnocom self._mutation_rate.
       Mutacija se vrsi nad jednim genom (karakterom) sa proizvoljnim indeksom.
    '''
    def mutate(self, genetic_code):
        random_value = random.random()
        
        # ukoliko je ispunjen uslov, izvrsi mutaciju
        if random_value < self.mutation_rate:
            
            # izabrati proizvoljan indeks
            random_index = random.randrange(self.chromosome_size)
            
            while True:
                # izabrati novu proizvoljnu vrednost za karakter
                new_value = random.choice(self.possible_gene_values)
                
                # ukoliko su vrednosti razlicite, izmeni karakter
                if genetic_code[random_index] != new_value:
                    break
                    
            genetic_code[random_index] = new_value
            
        return genetic_code
        
    '''Od jedinki generise novu generaciju primenjujuci genetske operatore 
       ukrstanje (crossover) i mutaciju (mutation). 
    '''
    def create_generation(self, chromosomes):
        generation = []
        generation_size = 0
        
        while generation_size < self.generation_size:
            # Proizvoljno se biraju 2 roditelja za ukrstanje
            [parent1, parent2] = random.sample(chromosomes, 2)
            
            # Dobijaju se 2 detata ukrstanjem
            child1_code, child2_code = self.crossover(parent1, parent2)
            
            # Vrsi se mutacija nad decom
            child1_code = self.mutate(child1_code)
            child2_code = self.mutate(child2_code)
            
            # Prave se novi hromozomi
            child1 = Chromosome(child1_code, self.calculate_fitness(child1_code))
            child2 = Chromosome(child2_code, self.calculate_fitness(child2_code))
            
            # Dodaju se u generaciju
            generation.append(child1)
            generation.append(child2)
            
            generation_size += 2
            
        return generation
            
    '''Jednopoziciono ukrstanje sa nasumicnom tackom ukrstanja'''
    def crossover(self, parent1, parent2):
        
        # bira se proizvoljna tacka ukrstanja
        break_point = random.randrange(1, self.chromosome_size)
        
        child1 = parent1.genetic_code[:break_point] + parent2.genetic_code[break_point:]
        child2 = parent2.genetic_code[:break_point] + parent1.genetic_code[break_point:]
        
        return (child1, child2)
        
    '''Izvrsavanje genetskog algoritma'''
    def optimize(self):
        # Generisi pocetnu populaciju jedinki i izracunaj
        # prilagodjenost svake jedinke u populaciji
        population = self.initial_population()
        
        for i in range(0, self.max_iterations):
            # Selekcija (ruletska ili turnirska)
            selected = self.selection(population)
            
            # Kreiraj generaciju ukrstanjem i mutacijom
            population = self.create_generation(selected)
            
            # Najkvalitetnija jedinka
            global_best_chromosome = max(population, key=lambda x: x.fitness)
                
            print(global_best_chromosome)
            
            if global_best_chromosome.fitness == self.chromosome_size:
                break
            
        return global_best_chromosome
            

genetic_algorithm = GeneticAlgorithm(string.ascii_letters, 'ABCDEFGHIJKL')
result = genetic_algorithm.optimize()

print('Result: {}'.format(result))

FBQDCFPoIJTx = 5
AHCgEAGqfqKL = 6
NBCgEAGHIJKw = 8
ABCgEAGHIJKL = 10
ABCDEFGYISKL = 10
ABCDCFGHIJKL = 11
ABCDEFGHIJKL = 12
Result: ABCDEFGHIJKL = 12
